# 使用 lxml 中的 xpath 提取文本与标签属性值

In [1]:
myPage = '''<html>
    <title>TITLE</title>
    <body>
    <h1>我的博客</h1>
    <div>我的文章</div>
    <div id="photos">
     <img src="pic1.jpeg"/><span id="pic1">PIC1 is beautiful!</span>
     <img src="pic2.jpeg"/><span id="pic2">PIC2 is beautiful!</span>
     <p><a href="http://www.example.com/more_pic.html">更多美图</a></p>
     <a href="http://www.baidu.com">去往百度</a>
     <a href="http://www.163.com">去往网易</a>
     <a href="http://www.sohu.com">去往搜狐</a>
    </div>
    <p class="myclassname">Hello,\nworld!<br/>-- by Adam</p>
    <div class="foot">放在尾部的其他一些说明</div>
    </body>
    </html>
'''

In [59]:
from lxml import etree
html = etree.fromstring(myPage)
print(html)

<Element html at 0x7fe652c32c88>


## 一、定位

In [60]:
html.xpath('//div')


[<Element div at 0x7fe652c9fd48>,
 <Element div at 0x7fe652c32908>,
 <Element div at 0x7fe652c32dc8>]

In [61]:
html.xpath('//div[@id]')  # 第二个

[<Element div at 0x7fe652c32908>]

In [63]:
html.xpath('//div[@class="foot"]')  # 第三个

[<Element div at 0x7fe652c32dc8>]

In [64]:
html.xpath('//div[@*]')  # 第二、三个

[<Element div at 0x7fe652c32908>, <Element div at 0x7fe652c32dc8>]

In [65]:
html.xpath('//div[1]')  # 第一个

[<Element div at 0x7fe652c9fd48>]

In [37]:
html.xpath('//div[last()-1]')  # 倒数第二个

[<Element div at 0x7fe652c82f08>]

In [38]:
html.xpath('//div[position()<3]')  # 第一、二个

[<Element div at 0x7fe6536c8a08>, <Element div at 0x7fe652c82f08>]

In [39]:
html.xpath('//div|//h1')  # 全部，另外包含h1

[<Element h1 at 0x7fe652c29ec8>,
 <Element div at 0x7fe6536c8a08>,
 <Element div at 0x7fe652c82f08>,
 <Element div at 0x7fe652c82508>]

In [41]:
html.xpath('//div[not(@*)]')  # 排除含有属性的

[<Element div at 0x7fe6536c8a08>]

## 二、取文本 text() 区别 html.xpath('string()')

In [66]:
html.xpath('//div/text()')

['我的文章',
 '\n     ',
 '\n     ',
 '\n     ',
 '\n     ',
 '\n     ',
 '\n     ',
 '\n    ',
 '放在尾部的其他一些说明']

In [67]:
html.xpath('//div[@id]/text()')

['\n     ', '\n     ', '\n     ', '\n     ', '\n     ', '\n     ', '\n    ']

In [68]:
html.xpath('//div[@class="foot"]/text()')

['放在尾部的其他一些说明']

In [69]:
html.xpath('//div[@*]/text()')

['\n     ',
 '\n     ',
 '\n     ',
 '\n     ',
 '\n     ',
 '\n     ',
 '\n    ',
 '放在尾部的其他一些说明']

In [70]:
html.xpath('//div[1]/text()')

['我的文章']

In [71]:
html.xpath('//div[last()-1]/text()')

['\n     ', '\n     ', '\n     ', '\n     ', '\n     ', '\n     ', '\n    ']

In [72]:
html.xpath('//div[position()<3]/text()')

['我的文章',
 '\n     ',
 '\n     ',
 '\n     ',
 '\n     ',
 '\n     ',
 '\n     ',
 '\n    ']

In [73]:
html.xpath('//div/text()|//h1/text()')

['我的博客',
 '我的文章',
 '\n     ',
 '\n     ',
 '\n     ',
 '\n     ',
 '\n     ',
 '\n     ',
 '\n    ',
 '放在尾部的其他一些说明']

## 三、取属性 @

In [74]:
html.xpath('//a/@href')

['http://www.example.com/more_pic.html',
 'http://www.baidu.com',
 'http://www.163.com',
 'http://www.sohu.com']

In [75]:
html.xpath('//img/@src')

['pic1.jpeg', 'pic2.jpeg']

In [76]:
html.xpath('//div[2]/span/@id')

['pic1', 'pic2']

## 四、定位（进阶）

In [77]:
divs = html.xpath('//div[position()<3]')
for div in divs:
    ass = div.findall('a')  # 这里只能找到:div->a, 找不到:div->p->a
    for a in ass:
        if a is not None:
            #print(dir(a))
            print(a.text, a.attrib.get('href')) #文档(DOM)元素(Element)的属性：text, attrib

去往百度 http://www.baidu.com
去往网易 http://www.163.com
去往搜狐 http://www.sohu.com


In [78]:
html.xpath('//div[position()<3]/a/@href')  # 只含第一层a

['http://www.baidu.com', 'http://www.163.com', 'http://www.sohu.com']

In [79]:
html.xpath('//div[position()<3]//a/@href')  # 包含多层的a

['http://www.example.com/more_pic.html',
 'http://www.baidu.com',
 'http://www.163.com',
 'http://www.sohu.com']

## 完